In [16]:
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [17]:
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10)
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11)
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9)

In [19]:
user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # (31176,4697)
user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # (31176,5834)
user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # (31176, 3250)

In [322]:
drop_user_place = pd.read_csv('../Data/whyout_data/drop_user_place.csv') # (22420,4697) 
drop_user_product = pd.read_csv('../Data/whyout_data/drop_user_product.csv') # (2996,5834)
drop_user_video = pd.read_csv('../Data/whyout_data/drop_user_video.csv') # (11067, 3250)

In [370]:
user = pd.read_csv('../Data/whyout_data/user.csv') # (31176,3)
drop_user_place_idx = pd.read_csv('../Data/whyout_data/drop_user_place_idx.csv') # (31176,4697)
drop_user_product_idx = pd.read_csv('../Data/whyout_data/drop_user_product_idx.csv') # (31176,5834)
drop_user_video_idx = pd.read_csv('../Data/whyout_data/drop_user_video_idx.csv') # (31176, 3250)
common_drop_user_idx = pd.read_csv('../Data/whyout_data/common_drop_user_idx.csv') # (25827,3)

In [214]:
common_drop_user_place = pd.read_csv('../Data/whyout_data/common_drop_user_place.csv') # (25827,4697)
common_drop_user_product = pd.read_csv('../Data/whyout_data/common_drop_user_product.csv') # (25827,5834)
common_drop_user_video = pd.read_csv('../Data/whyout_data/common_drop_user_video.csv') # (25827, 3250)

In [ ]:
drop_user_place_k40epochs1000 = pd.read_csv('../Data/whyout_data/drop_user_place_k40epochs1000.csv') # (22420,4697) 
drop_user_product_k20epochs1000 = pd.read_csv('../Data/whyout_data/drop_user_product.csv') # (2996,5834)
drop_user_video_k30epochs1000 = pd.read_csv('../Data/whyout_data/drop_user_video.csv') # (11067, 3250)

In [49]:
sgd_place_df_full = pd.read_csv('../Data/whyout_data/sgd_result/full_data/place_epochs2000.csv', index_col=0)
sgd_product_df_full = pd.read_csv('../Data/whyout_data/sgd_result/full_data/product_epochs2000.csv', index_col=0)
sgd_video_df_full = pd.read_csv('../Data/whyout_data/sgd_result/full_data/video_epochs2000.csv', index_col=0)

In [20]:
# 사용자의 평가 유무 확인
def find_zero_indices(df):
    return df.index[df.eq(0).all(axis=1)].tolist()

place_zero_indices = find_zero_indices(user_place)
product_zero_indices = find_zero_indices(user_product)
video_zero_indices = find_zero_indices(user_video)

common_zero_indices = set(place_zero_indices) & set(product_zero_indices) & set(video_zero_indices)
#place_product_zero_indices = set(place_zero_indices) & set(product_zero_indices)
#place_video_zero_indices = set(place_zero_indices) & set(video_zero_indices)
#product_video_zero_indices = set(product_zero_indices) & set(video_zero_indices)

print('장소를 평가하지 않은 사용자 수:',len(place_zero_indices))
print('상품을 평가하지 않은 사용자 수:',len(product_zero_indices))
print('영상을 평가하지 않은 사용자 수:',len(video_zero_indices))
print('모두 평가하지 않은 사용자 수:',len(common_zero_indices))
#print('장소 & 상품을 같이 평가하지 않은 사용자 수:',len(place_product_zero_indices))
#print('장소 & 영상을 같이 평가하지 않은 사용자 수:',len(place_video_zero_indices))
#print('상품 & 영상을 같이 평가하지 않은 사용자 수:',len(product_video_zero_indices))

장소를 평가하지 않은 사용자 수: 8756
상품을 평가하지 않은 사용자 수: 28180
영상을 평가하지 않은 사용자 수: 20109
모두 평가하지 않은 사용자 수: 5349


In [29]:
sgd_place_df = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_place_k50epochs1000.csv')
sgd_product_df = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_product_k20epochs1000.csv')
sgd_video_df = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_video_k40epochs1000.csv')

In [354]:
def recommend_items(df_sgd_preds, user_id, item_df, ratings_df, idx,num_recommendations):
    """
    :param df_sgd_preds: SGD 예측 평점 데이터프레임
    :param user_id: 사용자 ID
    :param df: 아이템 데이터프레임
    :param ratings_df: 사용자 평점 데이터프레임
    :param num_recommendations: 추천할 아이템 수
    :return: 사용자가 이미 평가한 아이템 인덱스와 추천 아이템 데이터프레임
    """
    if user_id in idx['idx'].values:
        userid = int(idx[idx['idx'] == user_id]['idx'].values[0])
        print(userid)
        matching_values = int(idx[idx['index'] == (user_id)]['idx'].values[0])
        print(matching_values)
    else:
        print('유저의 행동이 없습니다.')
        return None, None

    # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
    user_row_number = user_id - 1
    user_data = ratings_df.loc[user_row_number]
    #print(user_data)

    # 사용자가 이미 평가한 상품의 인덱스를 추출
    user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
    print(len(user_history_indices),user_history_indices)

    # SGD를 통해 예측된 사용자의 상품 평점을 기반으로 상품 데이터를 정렬
    sorted_user_predictions = df_sgd_preds.iloc[user_row_number].sort_values(ascending=False)
    #print(df_sgd_preds.iloc[user_row_number])

    # 사용자가 아직 평가하지 않은 상품 목록을 추출
    recommendations = item_df.loc[~item_df.index.isin(user_history_indices)]

    # 예측된 상품 평점 데이터와 원본 상품 데이터를 합침
    predictions_df = pd.DataFrame(sorted_user_predictions).reset_index()
    predictions_df['index'] = predictions_df['index'].astype(int)
    predictions_df.set_index('index', inplace=True)

    # recommendations의 인덱스와 predictions_df의 인덱스를 기준으로 병합
    recommendations = recommendations.merge(predictions_df, left_index=True, right_index=True)

    # 컬럼 이름을 바꾸고 정렬해서 반환
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations]
    print(f"user {matching_values}에게 추천해줄 {num_recommendations}개 아이템 id : {recommendations['idx'].values}")
    return user_history_indices, recommendations

In [373]:
# common_drop_user_place
already_rated, predictions = recommend_items(sgd_place_df_full, 15, place, common_drop_user_place, common_drop_user_idx,10)

유저의 행동이 없습니다.


In [374]:
already_rated1, predictions1 = recommend_items(sgd_place_df, 15, place, drop_user_place, drop_user_place_idx,10)
# place 10 : user idx 37
# place 7 : user idx 22

유저의 행동이 없습니다.
